# Ejercicio 13: LangChain

LangChain es un _framework_ de código abierto diseñado para facilitar el desarrollo de aplicaciones que combinan modelos de lenguaje LLMs con datos, herramientas externas y memoria. Está especialmente pensado para construir aplicaciones complejas basadas en IA, como sistemas _Retrieval-Augmented Generation_, asistentes conversacionales inteligentes, agentes autónomos y sistemas con razonamiento compuesto.

## Parte 1: Carga y preprocesamiento del corpus

In [44]:
%pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Danna\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [45]:
%pip install kaggle

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Danna\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [46]:
import os
from dotenv import load_dotenv
import kaggle
import pandas as pd

# Cargar variables de entorno desde .env
load_dotenv()

os.environ["KAGGLE_USERNAME"] = os.getenv("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = os.getenv("KAGGLE_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# Descargar el dataset (si no existe)
dataset = "rajneesh231/lex-fridman-podcast-transcript"
path = "../data/13langchain"

kaggle.api.dataset_download_files(dataset, path=path, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/rajneesh231/lex-fridman-podcast-transcript


In [47]:
#from google.colab import drive
#drive.mount('/content/drive')

In [48]:
# Cargar el dataset CSV
file_path = os.path.join(path, "podcastdata_dataset.csv")  # ajusta nombre si difiere
df = pd.read_csv(file_path)
df

,id,guest,title,text
0,1,Max Tegmark,Life 3.0,"As part of MIT course 6S099, Artificial Genera..."
1,2,Christof Koch,Consciousness,As part of MIT course 6S099 on artificial gene...
2,3,Steven Pinker,AI in the Age of Reason,"You've studied the human mind, cognition, lang..."
3,4,Yoshua Bengio,Deep Learning,What difference between biological neural netw...
4,5,Vladimir Vapnik,Statistical Learning,The following is a conversation with Vladimir ...
...,...,...,...,...
314,321,Ray Kurzweil,"Singularity, Superintelligence, and Immortality","By the time he gets to 2045, we'll be able to ..."
315,322,Rana el Kaliouby,"Emotion AI, Social Robots, and Self-Driving Cars","there's a broader question here, right? As we ..."
316,323,Will Sasso,"Comedy, MADtv, AI, Friendship, Madness, and Pr...",Once this whole thing falls apart and we are c...
317,324,Daniel Negreanu,Poker,you could be the seventh best player in the wh...


In [49]:
from langchain.schema import Document

# Convertir cada fila en un Document
documents = [
    Document(
        page_content=row["text"],
        metadata={"id": row["id"], "guest": row["guest"], "title": row["title"]},
    )
    for _, row in df.iterrows()
]

## Parte 2: Segmentación y embeddings

In [50]:
%pip install -U langchain-community

Defaulting to user installation because normal site-packages is not writeable
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully uninstalled pydantic-1.10.13
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 0.0.8 requires langchain-core<0.2,>=0.1, but you have langchain-core 0.3.71 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Danna\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [51]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

In [52]:
pip install faiss-cpu


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Danna\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [53]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
n=4000
texts = [chunk.page_content for chunk in chunks[:n]]
vectorstore = FAISS.from_texts(texts, embeddings)

vectorstore.save_local("index_13langchain")

## Parte 3: Indexación en FAISS

In [54]:
vectorstore = FAISS.from_texts(texts, embeddings)
vectorstore.save_local("index_13langchain_02")

In [55]:
!pip install protobuf==4.25.3
!pip install google-generativeai==0.3.2
!pip install proto-plus==1.23.0


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Danna\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Danna\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Danna\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [56]:
import google.generativeai as genai
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
for m in genai.list_models():
    print(m.name, m.supported_generation_methods)

models/embedding-gecko-001 ['embedText', 'countTextTokens']
models/gemini-1.0-pro-vision-latest ['generateContent', 'countTokens']
models/gemini-pro-vision ['generateContent', 'countTokens']
models/gemini-1.5-pro-latest ['generateContent', 'countTokens']
models/gemini-1.5-pro-002 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro ['generateContent', 'countTokens']
models/gemini-1.5-flash-latest ['generateContent', 'countTokens']
models/gemini-1.5-flash ['generateContent', 'countTokens']
models/gemini-1.5-flash-002 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-flash-8b ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-1.5-flash-8b-001 ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-1.5-flash-8b-latest ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-2.5-pro-preview-03-25 ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/ge

## Parte 4: Creación de la cadena de recuperación

In [57]:

pip install pydantic==1.10.13


Defaulting to user installation because normal site-packages is not writeable
  Using cached pydantic-1.10.13-py3-none-any.whl.metadata (149 kB)
Using cached pydantic-1.10.13-py3-none-any.whl (158 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.7
    Uninstalling pydantic-2.11.7:
      Successfully uninstalled pydantic-2.11.7
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.26 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.13 which is incompatible.
langchain-core 0.3.71 requires pydantic>=2.7.4, but you have pydantic 1.10.13 which is incompatible.
langchain-google-genai 0.0.8 requires langchain-core<0.2,>=0.1, but you have langchain-core 0.3.71 which is incompatible.
pydantic-settings 2.10.1 requires pydantic>=2.7.0, but you have pydantic 1.10.13 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Danna\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [58]:
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI

# Inicializar Gemini LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", convert_system_message_to_human=True)

# Crear retriever desde tu índice vectorial (FAISS o Chroma)
retriever = vectorstore.as_retriever()

# Cadena de pregunta-respuesta
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)

# Ejecutar una pregunta
response = qa_chain.run("What is AGI Artificial General Intelligence?")
print(response)

Based on the context, AGI or Artificial General Intelligence is:
*   Something to do with the "ability to learn and to adapt to new environments."
*   It is described as "something that resembles human level intelligence."
*   It would be able to "do exactly everything we humans can do better."
*   An AGI system might be one that "can ace the Turing test, natural language."


In [59]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Usa solo el siguiente contexto para responder a la pregunta.
Si la respuesta no está explícita en el contexto, responde exactamente:
"No encontré información suficiente en el corpus."

Contexto:
{context}

Pregunta: {question}
Respuesta:
"""
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

result = qa_chain.invoke({"query": "¿Qué es AGI Artificial General Intelligence?"})
print(result["result"])

Roughly, AGI es algo que tiene que ver con la capacidad de aprender y de adaptarse a nuevos entornos.


In [60]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    return_source_documents=True
)

result = qa_chain.invoke({"query": "¿Qué es AGI Artificial General Intelligence?"})
print(result["result"])
print(result["source_documents"])

Based on the context, Artificial General Intelligence (AGI) is:

*   Often considered "not very well defined."
*   Roughly described as having the "ability to learn and to adapt to new environments."
*   A "radical difference from experience" (in comparison to narrow AI).
*   Not necessarily conscious; a machine with AGI "may or may not" have consciousness.
*   Hypothesized by some to ultimately be a "simple one," possibly describable by a "pseudocode of a few lines."
[Document(page_content="bit about the Turing test, StarCraft. All of these have echoes of general intelligence. But if you think about just something that you would sit back and say, wow, this is really something that resembles human level intelligence. What do you think it takes to build that? So I find that AGI oftentimes is maybe not very well defined. So what I'm trying to then come up with for myself is what would be a result look like that you would start to believe that you would have agents or neural nets that"), 